In [1]:
# External modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
import numpy as np
import os, sys
import json

pd.options.mode.chained_assignment = None

In [2]:
with open("../clientid.json", "r") as file:
    secrets = json.load(file)

os.environ["API_CLIENT_ID"] = secrets["API_CLIENT_ID"]
os.environ["API_CLIENT_SECRET"] = secrets["API_CLIENT_SECRET"]

In [3]:
import refrunrank as rrr
import oms

In [4]:
importlib.reload(rrr)
importlib.reload(oms)

CHData = rrr.CHRunData("../jsons/AllRunsRefRuns_Mar5_2024.json", "../jsons/json_TRK_PromptReco_2018Runs.json")
oms_fetch = oms.oms_fetch()

In [5]:
CHData.goldenRunsDF

,run_number,run_reconstruction_type,reference_run_number,reference_run_reconstruction_type,dataset
0,313098,express,312727,express,To Be Defined
1,313106,express,312727,express,To Be Defined
2,313108,express,312727,express,To Be Defined
3,313118,express,312727,express,To Be Defined
4,313120,express,312727,express,To Be Defined
...,...,...,...,...,...
3851,325101,rerecoul,320804,prompt,/ReReco/Run2018D_UL2019/DQM
3852,325117,rerecoul,320804,prompt,/ReReco/Run2018D_UL2019/DQM
4080,316469,prompt,316457,prompt,/ZeroBias/Run2018A-PromptReco-v2/DQMIO
4081,323701,prompt,321067,prompt,/PromptReco/Collisions2018D/DQM


## Checking setup

In [6]:
# Using the API, you can check all of the available features
print("All available run level features:\n")
oms_fetch.get_oms_data(308997, "runs")

All available run level features:

https://vocms0185.cern.ch/agg/api/v1/runs/?filter[run_number][GE]=308997&filter[run_number][LE]=308997&page[offset]=0&page[limit]=1


,index,components,l1_hlt_mode_stripped,l1_key,duration,b_field,tier0_transfer,l1_triggers_counter,hlt_physics_throughput,init_lumi,...,fill_type_runtime,hlt_key,clock_type,last_lumisection_number,l1_rate,l1_menu,run_number,stable_beam,beta_star_stop,hlt_physics_counter
0,0,"[DAQ, ECAL, TCDS]",None,None,61,0.019,False,None,None,None,...,None,/minidaq/EcalCalibration/2017/HLT/V4,LOCAL,None,None,None,308997,False,None,None


In [7]:
oms_fetch.get_oms_data([313118, 355408], "runs")

https://vocms0185.cern.ch/agg/api/v1/runs/?filter[run_number][EQ]=313118&page[offset]=0&page[limit]=10
https://vocms0185.cern.ch/agg/api/v1/runs/?filter[run_number][EQ]=355408&page[offset]=0&page[limit]=10


,index,components,l1_hlt_mode_stripped,l1_key,duration,b_field,tier0_transfer,l1_triggers_counter,hlt_physics_throughput,init_lumi,...,fill_type_runtime,hlt_key,clock_type,last_lumisection_number,l1_rate,l1_menu,run_number,stable_beam,beta_star_stop,hlt_physics_counter
0,0,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",cosmics2018/v85,l1_trg_cosmics2018/v50,3353,3.801,True,2760947,0.014939,0.0,...,COSMICS,/cdaq/cosmic/commissioning2018/CRAFT/v1.0/HLT/V1,LOCAL,143,828.286,L1Menu_Collisions2017_v4,313118,False,<NA>,487747
1,1,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",collisions2022/v94,l1_trg_collisions2022/v23,813,3.8,False,70454612,17.182108,0.275994,...,PROTONS,/cdaq/physics/firstCollisions22/v4.3/HLT/V8,LHC,33,93477.5,L1Menu_Collisions2022_v1_2_0,355408,True,30,4496306


In [8]:
oms_fetch.get_oms_data((313118, 320000), "runs")

https://vocms0185.cern.ch/agg/api/v1/runs/?filter[run_number][GE]=313118&filter[run_number][LE]=318117&page[offset]=0&page[limit]=5000
https://vocms0185.cern.ch/agg/api/v1/runs/?filter[run_number][GE]=318118&filter[run_number][LE]=320000&page[offset]=0&page[limit]=5000


,index,components,l1_hlt_mode_stripped,l1_key,duration,b_field,tier0_transfer,l1_triggers_counter,hlt_physics_throughput,init_lumi,...,fill_type_runtime,hlt_key,clock_type,last_lumisection_number,l1_rate,l1_menu,run_number,stable_beam,beta_star_stop,hlt_physics_counter
0,0,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",cosmics2018/v85,l1_trg_cosmics2018/v50,3353,3.801,True,2760947,0.014939,0.0,...,COSMICS,/cdaq/cosmic/commissioning2018/CRAFT/v1.0/HLT/V1,LOCAL,143,828.286,L1Menu_Collisions2017_v4,313118,False,<NA>,487747
1,1,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",cosmics2018/v85,l1_trg_cosmics2018/v50,216,3.801,True,<NA>,<NA>,<NA>,...,COSMICS,<NA>,LOCAL,<NA>,<NA>,L1Menu_Collisions2017_v4,313119,False,<NA>,<NA>
2,2,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",cosmics2018/v85,l1_trg_cosmics2018/v50,3633,3.801,True,2992672,0.014794,0.0,...,COSMICS,/cdaq/cosmic/commissioning2018/CRAFT/v1.0/HLT/V1,LOCAL,155,828.296,L1Menu_Collisions2017_v4,313120,False,<NA>,527159
3,3,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",cosmics2018/v85,l1_trg_cosmics2018/v50,64,3.801,True,<NA>,<NA>,<NA>,...,COSMICS,<NA>,LOCAL,<NA>,<NA>,L1Menu_Collisions2017_v4,313121,False,<NA>,<NA>
4,4,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",cosmics2018/v85,l1_trg_cosmics2018/v50,214,3.801,True,<NA>,<NA>,<NA>,...,COSMICS,<NA>,LOCAL,<NA>,<NA>,L1Menu_Collisions2017_v4,313122,False,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4226,1237,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",collisions2018/v169,l1_trg_collisions2018/v43,6186,3.8,True,394654283,0.651722,1.140351,...,PROTONS,/cdaq/physics/Run2018/2e34/v3.1.0/HLT/V1,LHC,264,64621.008,L1Menu_Collisions2018_v2_0_0,319992,True,30,5626515
4227,1238,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",collisions2018/v169,l1_trg_collisions2018/v43,22539,3.8,True,1486183334,0.432615,0.945159,...,PROTONS,/cdaq/physics/Run2018/2e34/v3.1.0/HLT/V1,LHC,965,66069.83,L1Menu_Collisions2018_v2_0_0,319993,True,30,16283980
4228,1239,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",cosmics2018/v154,l1_trg_cosmics2018/v99,1931,3.8,True,1303692,0.015889,0.0,...,COSMICS,/cdaq/cosmic/commissioning2018/CRAFT/v1.3/HLT/V1,LOCAL,82,682.055,L1Menu_Collisions2018_v1_0_0,319995,False,<NA>,397834
4229,1240,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",circulating2018/v81,l1_trg_circulating2018/v48,2927,3.8,True,1682688,0.006603,0.0,...,PROTONS,/cdaq/special/Circulating2018/v1.0.4/HLT/V1,LHC,124,582.157,L1Menu_Collisions2018_v1_0_0,319998,False,<NA>,263602


In [14]:
filters = {
    # "run_number": (313118, 377251),
    # "reference_run_number": 312727,
    # "run_reconstruction_type": "express",
    # "reference_run_reconstruction_type": "express",
    "dataset": "/Cosmics*"
}

CHData.applyfilter(filters=filters)

,run_number,run_reconstruction_type,reference_run_number,reference_run_reconstruction_type,dataset
10,313130,prompt,313105,prompt,/Cosmics/Commissioning2018-PromptReco-v1/DQMIO
57,319530,prompt,317464,prompt,/Cosmics/Run2018C-PromptReco-v2/DQMIO
63,314850,prompt,314768,prompt,/Cosmics/Commissioning2018-PromptReco-v1/DQMIO
113,314881,prompt,314768,prompt,/Cosmics/Commissioning2018-PromptReco-v1/DQMIO
114,314883,prompt,314768,prompt,/Cosmics/Commissioning2018-PromptReco-v1/DQMIO
...,...,...,...,...,...
3420,325927,prompt,323586,prompt,/Cosmics/HIRun2018-PromptReco-v1/DQMIO
3421,325924,prompt,323586,prompt,/Cosmics/HIRun2018-PromptReco-v1/DQMIO
3422,325919,prompt,323586,prompt,/Cosmics/HIRun2018-PromptReco-v1/DQMIO
3423,325913,prompt,323586,prompt,/Cosmics/HIRun2018-PromptReco-v1/DQMIO


In [11]:
CHData.RunsDF

,run_number,run_reconstruction_type,reference_run_number,reference_run_reconstruction_type,dataset
0,313098,express,312727,express,To Be Defined
1,313106,express,312727,express,To Be Defined
2,313108,express,312727,express,To Be Defined
3,313118,express,312727,express,To Be Defined
4,313120,express,312727,express,To Be Defined
...,...,...,...,...,...
10443,377251,prompt,377127,prompt,/PromptReco/Cosmics2024/DQM
10444,377254,prompt,377127,prompt,/PromptReco/Cosmics2024/DQM
10445,377255,prompt,377127,prompt,/PromptReco/Cosmics2024/DQM
10446,377295,prompt,377127,prompt,/PromptReco/Cosmics2024/DQM


# User Input

In [9]:
importlib.reload(rrr)
importlib.reload(oms)

CHData = rrr.CHRunData("../jsons/AllRunsRefRuns_Mar5_2024.json", "../jsons/json_TRK_PromptReco_2018Runs.json")
oms_fetch = oms.oms_fetch()

In [10]:
# User input: Features that will be used 
ftrs_lst = [
    'b_field',
    'init_lumi',
    'energy',
    'end_lumi',
    'hlt_physics_rate',
    'l1_rate',
    'run_number',
]

# Target run
target = 316469

# If testing or not
testing = True

# Getting run numbers to query
CHfilters = {
    "run_number": (target-500, target),
    # "reference_run_number": 312727,
    "run_reconstruction_type": "prompt",
    # "reference_run_reconstruction_type": "express",
    "dataset": "/ZeroBias/Run2018*"
}

CHData.applyfilter(filters=CHfilters)
candsrunnb = list(CHData.filteredDF["run_number"].unique())

# if target in candsrunnb:
    # candsrunnb.remove(target)

# OMS Filter
min_lss = 500

# Filters that will be applied in query to OMS
oms_filter = [
    {"attribute_name": "last_lumisection_number", "value": min_lss, "operator": "GE"},
]

# Fetching Data

In [11]:
# Getting data from oms
oms_fetch.get_oms_data(candsrunnb, "runs", extrafilters=oms_filter)

https://vocms0185.cern.ch/agg/api/v1/runs/?filter[run_number][EQ]=315974&filter[last_lumisection_number][GE]=500&page[offset]=0&page[limit]=10
https://vocms0185.cern.ch/agg/api/v1/runs/?filter[run_number][EQ]=315973&filter[last_lumisection_number][GE]=500&page[offset]=0&page[limit]=10
https://vocms0185.cern.ch/agg/api/v1/runs/?filter[run_number][EQ]=316058&filter[last_lumisection_number][GE]=500&page[offset]=0&page[limit]=10
https://vocms0185.cern.ch/agg/api/v1/runs/?filter[run_number][EQ]=316060&filter[last_lumisection_number][GE]=500&page[offset]=0&page[limit]=10
https://vocms0185.cern.ch/agg/api/v1/runs/?filter[run_number][EQ]=316061&filter[last_lumisection_number][GE]=500&page[offset]=0&page[limit]=10
https://vocms0185.cern.ch/agg/api/v1/runs/?filter[run_number][EQ]=316062&filter[last_lumisection_number][GE]=500&page[offset]=0&page[limit]=10
https://vocms0185.cern.ch/agg/api/v1/runs/?filter[run_number][EQ]=316111&filter[last_lumisection_number][GE]=500&page[offset]=0&page[limit]=10

,index,components,l1_hlt_mode_stripped,l1_key,duration,b_field,tier0_transfer,l1_triggers_counter,hlt_physics_throughput,init_lumi,...,fill_type_runtime,hlt_key,clock_type,last_lumisection_number,l1_rate,l1_menu,run_number,stable_beam,beta_star_stop,hlt_physics_counter
0,0,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",collisions2018/v94,l1_trg_collisions2018/v16,21347,3.801,True,1011355699,1.092859,1.807007,...,PROTONS,/cdaq/physics/Run2018/2e34/v1.2.3/HLT/V2,LHC,914,47469.637,L1Menu_Collisions2018_0_0_1,315973,True,30,27470119
1,1,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",collisions2018/v97,l1_trg_collisions2018/v17,21818,3.801,True,956062773,0.496014,1.122686,...,PROTONS,/cdaq/physics/Run2018/2e34/v2.0.1/HLT/V1,LHC,935,43866.527,L1Menu_Collisions2018_v1_0_0,316060,True,30,17147121
2,2,"[CSC, CTPPS, DAQ, DCS, DQM, DT, ECAL, ES, HCAL...",collisions2018/v104,l1_trg_collisions2018/v20,18297,3.801,True,1047191822,0.985871,0.650669,...,PROTONS,/cdaq/physics/Run2018/2e34/v2.0.3/HLT/V1,LHC,784,57301.848,L1Menu_Collisions2018_v1_0_0,316153,True,30,20969681
3,3,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",collisions2018/v104,l1_trg_collisions2018/v20,48456,3.801,True,2926463027,0.686344,1.788128,...,PROTONS,/cdaq/physics/Run2018/2e34/v2.0.3/HLT/V1,LHC,2077,60445.62,L1Menu_Collisions2018_v1_0_0,316187,True,30,44128512
4,4,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",collisions2018/v104,l1_trg_collisions2018/v20,27932,3.801,True,1652909745,1.042354,2.044305,...,PROTONS,/cdaq/physics/Run2018/2e34/v2.0.3/HLT/V1,LHC,1197,59289.25,L1Menu_Collisions2018_v1_0_0,316199,True,30,32668682
5,5,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",collisions2018/v104,l1_trg_collisions2018/v20,14634,3.801,True,792062135,1.045378,1.857649,...,PROTONS,/cdaq/physics/Run2018/2e34/v2.0.3/HLT/V1,LHC,626,54280.414,L1Menu_Collisions2018_v1_0_0,316239,True,30,17022640
6,6,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",collisions2018/v104,l1_trg_collisions2018/v20,28586,3.801,True,1731043211,0.548126,1.327663,...,PROTONS,/cdaq/physics/Run2018/2e34/v2.0.3/HLT/V1,LHC,1224,60771.64,L1Menu_Collisions2018_v1_0_0,316240,True,30,23257181
7,7,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",collisions2018/v104,l1_trg_collisions2018/v20,23888,3.801,True,1432849067,0.577878,1.326357,...,PROTONS,/cdaq/physics/Run2018/2e34/v2.0.3/HLT/V1,LHC,1023,60087.33,L1Menu_Collisions2018_v1_0_0,316218,True,30,20108982
8,8,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",collisions2018/v115,l1_trg_collisions2018/v20,28483,3.801,True,1064537461,0.690183,0.83586,...,PROTONS,/cdaq/physics/Run2018/2e34/v2.1.1/HLT/V1,LHC,1221,37403.22,L1Menu_Collisions2018_v1_0_0,316380,True,30,24359007
9,9,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",collisions2018/v119,l1_trg_collisions2018/v21,34137,3.801,True,1635592908,0.860118,1.338072,...,PROTONS,/cdaq/physics/Run2018/2e34/v2.1.2/HLT/V1,LHC,1463,47961.145,L1Menu_Collisions2018_v1_0_0,316457,True,30,35112933


In [12]:
# Creating RunRanker instance

importlib.reload(rrr)
import refrunrank as rrr

ranker = rrr.RunRanker(oms_fetch.last_run_query, ftrs_lst)

# Ranking

In [77]:
ranker.refrank(target)

,,PC1,PC2,dist,b_field (0.0),init_lumi (0.4094),energy (0.0),end_lumi (0.3365),hlt_physics_rate (0.1902),l1_rate (0.0568),run_number (0.0072)
rank,run,,,,,,,,,,
0,316469,-1.218802,1.590140,0.000000,3.801,1.70394,6499,1.183357,1150.782,52243.504,316469
1,316239,-1.930819,0.418794,1.370773,3.801,1.857649,6499,1.329343,1166.548,54280.414,316239
2,316457,0.592608,1.397763,1.821597,3.801,1.338072,6499,0.688284,1029.61,47961.145,316457
3,316199,-2.016427,-0.394900,2.139296,3.801,2.044305,6499,1.092212,1171.792,59289.25,316199
4,315973,-1.753906,-0.533645,2.190159,3.801,1.807007,6499,1.116081,1289.331,47469.637,315973
5,316187,-0.324117,-0.967040,2.709176,3.801,1.788128,6499,0.680513,911.45,60445.62,316187
6,316218,0.424601,-0.729371,2.842693,3.801,1.326357,6499,0.771696,843.266,60087.33,316218
7,316240,0.503095,-0.690042,2.857300,3.801,1.327663,6499,0.759214,816.462,60771.64,316240
8,316060,1.425083,-0.279388,3.238096,3.801,1.122686,6499,0.698668,786.737,43866.527,316060


# Testing the Ranking System

---